In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olaolabenjo/englishyoruba-parallel-corpus-sample")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'englishyoruba-parallel-corpus-sample' dataset.
Path to dataset files: /kaggle/input/englishyoruba-parallel-corpus-sample


In [2]:
import pandas as pd

df = pd.read_csv('/kaggle/input/englishyoruba-parallel-corpus-sample/yoruba_english_parallel_sample.csv')

# Assuming columns: "english", "yoruba"
english_sentences = df["english"].astype(str)
yoruba_sentences = df["yoruba"].astype(str)

english_sentences.to_csv("/content/sample_data/english.txt", index=False, header=False)
yoruba_sentences.to_csv("/content/sample_data/yoruba.txt", index=False, header=False)


In [3]:
## Build Vocabulary
from collections import Counter

def build_vocab(sentences: list, min_freq: int = 1):
    tokens = []
    for s in sentences:
        tokens.extend(s.lower().split())

    freq = Counter(tokens)
    vocab = {word: idx for idx, (word, c) in enumerate(freq.items()) if c >= min_freq}
    return vocab


In [4]:
## CBOW dataset

import torch
from torch.utils.data import Dataset

class CBOWDataset(Dataset):
    def __init__(self, sentences: list, vocab: dict, window: int =2):
        self.data = []
        self.vocab = vocab

        for s in sentences:
            words = s.lower().split()
            for i in range(window, len(words) - window):
                context = words[i-window:i] + words[i+1:i+window+1]
                target = words[i]

                if target in vocab and all(w in vocab for w in context):
                    self.data.append(
                        ([vocab[w] for w in context], vocab[target])
                    )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, target = self.data[idx]
        return torch.tensor(context), torch.tensor(target)

In [5]:
## CBOW model

import torch.nn as nn

class CBOW(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        embeds = self.emb(x)          # [B, context, D]
        mean = embeds.mean(dim=1)     # CBOW
        return self.linear(mean)


In [6]:
from torch.utils.data import DataLoader

def train_cbow(sentences, save_path: str, embedding_dim: int = 50, epochs: int = 1000):
    vocab = build_vocab(sentences)
    dataset = CBOWDataset(sentences, vocab)
    loader = DataLoader(dataset, batch_size=128, shuffle=True)

    model = CBOW(len(vocab), embed_dim=embedding_dim)
    loss_fn = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(epochs):
        for x, y in loader:
            loss = loss_fn(model(x), y)
            optim.zero_grad()
            loss.backward()
            optim.step()

        print(f"Epoch {epoch} Loss {loss.item():.4f}")

    torch.save({
        "embeddings": model.emb.weight.data,
        "vocab": vocab
    }, save_path)


In [7]:
## Build seq2seq model

class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers=3, batch_first=True)

    def forward(self, x):
        emb = self.embedding(x)
        outputs, (h, c) = self.lstm(emb)
        return h, c

class Decoder(nn.Module):
  def __init__(self, vocab_size, emb_dim, hidden_dim):
      super().__init__()
      self.embedding = nn.Embedding(vocab_size, emb_dim)
      self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers=3, batch_first=True)
      self.fc = nn.Linear(hidden_dim, vocab_size)

  def forward(self, x, h, c):
      emb = self.embedding(x)
      outputs, (h, c) = self.lstm(emb, (h, c))
      return self.fc(outputs), h, c



In [8]:
class seq2seqDataset(Dataset):
    def __init__(self, eng, yor, eng_vocab, yor_vocab):
        self.pairs = []
        for e, y in zip(eng, yor):
            e_tokens = e.lower().split()
            y_tokens = y.lower().split()

            # Filter for words in vocab and add <sos>, <eos>
            e_ids = [eng_vocab["<sos>"]] + \
                    [eng_vocab[w] for w in e_tokens if w in eng_vocab] + \
                    [eng_vocab["<eos>"]]

            y_ids = [yor_vocab["<sos>"]] + \
                    [yor_vocab[w] for w in y_tokens if w in yor_vocab] + \
                    [yor_vocab["<eos>"]]

            self.pairs.append((e_ids, y_ids))

    def __getitem__(self, idx):
        # Explicitly ensure LongTensor type for indices
        return torch.tensor(self.pairs[idx][0], dtype=torch.long), \
               torch.tensor(self.pairs[idx][1], dtype=torch.long)

    def __len__(self):
        return len(self.pairs)

In [9]:
## Train seq2seq model

def train_seq2seq(encoder, decoder, loader, epochs: int = 1000):
    loss_fn = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()))

    for epoch in range(epochs):
        for src, tgt in loader:
            h, c = encoder(src)
            output, _, _ = decoder(tgt[:, :-1], h, c)

            loss = loss_fn(
                output.reshape(-1, output.shape[-1]),
                tgt[:, 1:].reshape(-1)
            )

            optim.zero_grad()
            loss.backward()
            optim.step()

        print(f"Epoch {epoch} Loss {loss.item():.4f}")


In [10]:
## Function to translate english to yoruba

def translate(sentence, encoder, decoder, eng_vocab, yor_vocab, max_len=20):
    # Set models to evaluation mode
    encoder.eval()
    decoder.eval()
    inv_vocab = {v:k for k,v in yor_vocab.items()}

    src = torch.tensor([[eng_vocab[w] for w in sentence.lower().split() if w in eng_vocab]])
    h, c = encoder(src)

    token = torch.tensor([[yor_vocab["<sos>"]]])
    result = []

    for _ in range(max_len):
        out, h, c = decoder(token, h, c)
        idx = out.argmax(-1).item()
        if inv_vocab[idx] == "<eos>":
            break
        if inv_vocab[idx] == "<pad>":
            token = torch.tensor([[idx]])
            continue
        result.append(inv_vocab[idx])
        token = torch.tensor([[idx]])

    return " ".join(result)


In [11]:
def build_vocab_seq2seq(sentences, min_freq=1, specials=["<pad>", "<sos>", "<eos>"]):
    from collections import Counter

    counter = Counter()
    for s in sentences:
        counter.update(s.split())

    vocab = {tok: idx for idx, tok in enumerate(specials)}
    idx = len(specials)

    for word, freq in counter.items():
        if freq >= min_freq and word not in vocab:
            vocab[word] = idx
            idx += 1

    return vocab


In [12]:
## Build vocabulary for the seq2seq model

eng_vocab_seq2seq = build_vocab_seq2seq(english_sentences)
yor_vocab_seq2seq = build_vocab_seq2seq(yoruba_sentences)



In [13]:
## Use CBOW to learn embeddings and save them

train_cbow(open("/content/sample_data/english.txt").readlines(), "/content/sample_data/eng_embeddings.pt", embedding_dim=40, epochs = 1500)
train_cbow(open("/content/sample_data/yoruba.txt").readlines(), "/content/sample_data/yor_embeddings.pt", embedding_dim=40, epochs = 1500)

Epoch 0 Loss 7.1937
Epoch 1 Loss 6.9589
Epoch 2 Loss 7.1967
Epoch 3 Loss 6.9228
Epoch 4 Loss 6.6001
Epoch 5 Loss 6.6481
Epoch 6 Loss 6.5526
Epoch 7 Loss 6.5531
Epoch 8 Loss 6.3424
Epoch 9 Loss 6.0126
Epoch 10 Loss 6.4519
Epoch 11 Loss 6.4332
Epoch 12 Loss 6.1831
Epoch 13 Loss 6.4150
Epoch 14 Loss 6.1548
Epoch 15 Loss 6.2810
Epoch 16 Loss 5.9682
Epoch 17 Loss 5.4196
Epoch 18 Loss 5.5506
Epoch 19 Loss 5.8395
Epoch 20 Loss 5.4401
Epoch 21 Loss 5.4810
Epoch 22 Loss 3.9749
Epoch 23 Loss 5.3422
Epoch 24 Loss 4.3186
Epoch 25 Loss 4.5304
Epoch 26 Loss 5.5684
Epoch 27 Loss 5.3107
Epoch 28 Loss 4.8641
Epoch 29 Loss 5.3337
Epoch 30 Loss 5.6560
Epoch 31 Loss 5.0736
Epoch 32 Loss 5.0653
Epoch 33 Loss 4.9969
Epoch 34 Loss 4.8080
Epoch 35 Loss 4.3239
Epoch 36 Loss 4.1983
Epoch 37 Loss 4.0952
Epoch 38 Loss 4.0261
Epoch 39 Loss 4.0804
Epoch 40 Loss 3.3125
Epoch 41 Loss 3.7868
Epoch 42 Loss 4.4506
Epoch 43 Loss 4.2252
Epoch 44 Loss 3.7485
Epoch 45 Loss 4.3860
Epoch 46 Loss 2.9030
Epoch 47 Loss 3.4168
Ep

In [14]:
## Load embeddings
eng_embed_data = torch.load("/content/sample_data/eng_embeddings.pt")
yor_embed_data = torch.load("/content/sample_data/yor_embeddings.pt")

eng_embeddings = eng_embed_data["embeddings"]
yor_embeddings = yor_embed_data["embeddings"]

In [15]:
def encode_sentence(sentence, vocab):
    return [vocab["<sos>"]] + \
           [vocab[w] for w in sentence.lower().split() if w in vocab] + \
           [vocab["<eos>"]]

def collate_batch(batch):
    src_list, tgt_list = [], []
    for _src, _tgt in batch:
        src_list.append(_src)
        tgt_list.append(_tgt)

    # Pad sequences to the length of the longest sequence in the batch
    src_padded = torch.nn.utils.rnn.pad_sequence(src_list, batch_first=True, padding_value=eng_vocab_seq2seq["<pad>"])
    tgt_padded = torch.nn.utils.rnn.pad_sequence(tgt_list, batch_first=True, padding_value=yor_vocab_seq2seq["<pad>"])
    return src_padded, tgt_padded

## Create the seq2seq Dataset

dataset = seq2seqDataset(
    english_sentences,
    yoruba_sentences,
    eng_vocab_seq2seq,
    yor_vocab_seq2seq
)

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_batch
)

In [17]:
##Initialize the encoder and the decoder

encoder = Encoder(
    vocab_size=len(eng_vocab_seq2seq),
    emb_dim=40,
    hidden_dim=256
)

decoder = Decoder(
    vocab_size=len(yor_vocab_seq2seq),
    emb_dim=40,
    hidden_dim=256
)


## Load embeddings into the encoder and decoder

encoder.embedding.weight.data[:eng_embeddings.size(0)] = eng_embeddings
decoder.embedding.weight.data[:yor_embeddings.size(0)] = yor_embeddings

# Freeze embedding layers
encoder.embedding.weight.requires_grad = False
decoder.embedding.weight.requires_grad = False

In [18]:
## Train seq2seq model

train_seq2seq(
    encoder=encoder,
    decoder=decoder,
    loader=loader,
    epochs= 300
)


Epoch 0 Loss 3.7866
Epoch 1 Loss 4.2069
Epoch 2 Loss 2.7414
Epoch 3 Loss 4.3296
Epoch 4 Loss 2.5871
Epoch 5 Loss 3.2654
Epoch 6 Loss 2.9060
Epoch 7 Loss 3.3352
Epoch 8 Loss 3.7327
Epoch 9 Loss 3.8619
Epoch 10 Loss 4.6727
Epoch 11 Loss 4.2110
Epoch 12 Loss 3.6902
Epoch 13 Loss 2.4531
Epoch 14 Loss 2.4964
Epoch 15 Loss 1.9751
Epoch 16 Loss 1.8255
Epoch 17 Loss 2.5675
Epoch 18 Loss 2.8707
Epoch 19 Loss 2.1539
Epoch 20 Loss 3.4535
Epoch 21 Loss 4.1279
Epoch 22 Loss 2.1117
Epoch 23 Loss 2.5850
Epoch 24 Loss 2.6045
Epoch 25 Loss 2.6995
Epoch 26 Loss 2.9473
Epoch 27 Loss 3.3064
Epoch 28 Loss 1.8115
Epoch 29 Loss 2.5558
Epoch 30 Loss 3.2720
Epoch 31 Loss 2.4639
Epoch 32 Loss 3.0325
Epoch 33 Loss 2.3684
Epoch 34 Loss 3.1278
Epoch 35 Loss 2.6425
Epoch 36 Loss 1.9402
Epoch 37 Loss 2.9690
Epoch 38 Loss 2.3924
Epoch 39 Loss 1.4856
Epoch 40 Loss 1.6397
Epoch 41 Loss 2.4523
Epoch 42 Loss 2.3201
Epoch 43 Loss 3.5102
Epoch 44 Loss 2.1601
Epoch 45 Loss 3.0707
Epoch 46 Loss 2.7916
Epoch 47 Loss 2.4052
Ep

In [30]:
sentence = "If the devil could organize to fight in heaven"
translation = translate(
    sentence,
    encoder,
    decoder,
    eng_vocab_seq2seq,
    yor_vocab_seq2seq
)

print(translation)


bí ìmọ́lẹ̀ ṣe ń gbìyànjú láti bọ́ nínú rẹ̀ sí dí kò ní wáyé. tó.
